In [12]:
# Importing libraries
import os
import pandas as pd
import jellyfish
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import time

In [2]:
# Importing sample data
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
grandparent_dir = os.path.dirname(parent_dir)
data_dir = os.path.join(grandparent_dir, 'data/conflict/sample')
print(data_dir)

/Users/eshan23/eshanprashar_git_profile/judges-conflicts/data/conflict/sample


In [3]:
# Importing data
sample_dockets = pd.read_csv(os.path.join(data_dir, 'sampled_dockets.csv'))
print(f'The shape of the sample_dockets is {sample_dockets.shape}')
sample_investments = pd.read_csv(os.path.join(data_dir, 'sampled_investments.csv'))
print(f'The shape of the sample_investments is {sample_investments.shape}')

The shape of the sample_dockets is (5913, 10)
The shape of the sample_investments is (12242, 4)


In [6]:
# Examine dockets sample
sample_dockets.head()

,author_id,judge,docketNumber,docket_id,snippet,caseName,dateFiled,party_1,party_2,dateFiled_year
0,2490,Paine,91-6548-CIV.,691131,\n795 F.Supp. 1112 (1992)\nChristina Elisabeth...,Watchorn Ex Rel. Christenson v. Town of Davie,1992-08-07,Watchorn Ex Rel. Christenson,Town of Davie,1992
1,2490,Paine,"93-2123-CIV-PAINE, 94-1692-CIV-PAINE",2379325,"\n127 F.Supp.2d 1239 (1999)\nDICTIOMATIC, INC....","Dictiomatic, Inc. v. United States Fidelity & ...",1999-06-15,"Dictiomatic, Inc.",United States Fidelity & Guaranty Co.,1999
2,2490,Paine,99-10054-CIV,2371116,\n104 F.Supp.2d 1368 (2000)\nElizabeth J. NEUM...,Neumont v. Monroe County Florida,2000-06-21,Neumont,Monroe County Florida,2000
3,2490,Paine,98-10060-CIV.,2369251,"\n42 F.Supp.2d 1344 (1999)\nDeborah K. BRANDT,...","Brandt v. Weather Channel, Inc.",1999-03-18,Brandt,"Weather Channel, Inc.",1999
4,2490,Paine,99-10054-CIV-PAINE/VITUNAC,2368734,\n280 F.Supp.2d 1367 (2003)\nElizabeth J. NEUM...,"Neumont v. Monroe County, Florida",2003-05-21,Neumont,"Monroe County, Florida",2003


In [7]:
# From the columns above, we only need author_id, judge, docketNumber, docket_id, party_1, party_2, and dateFiled_year
columns_to_keep = ['author_id', 'judge', 'docketNumber', 'docket_id', 'caseName','party_1', 'party_2', 'dateFiled_year']
sample_dockets_reduced_v1 = sample_dockets[columns_to_keep]

# Because investment data is available for years 2002 onwards, we will only keep dockets from 2002 onwards
sample_dockets_reduced_v1 = sample_dockets_reduced_v1[sample_dockets_reduced_v1['dateFiled_year'] >= 2002]
print(f'The shape of the reduced dockets sample is {sample_dockets_reduced_v1.shape}')
sample_dockets_reduced_v1.head()

The shape of the reduced dockets sample is (2094, 8)


,author_id,judge,docketNumber,docket_id,caseName,party_1,party_2,dateFiled_year
4,2490,Paine,99-10054-CIV-PAINE/VITUNAC,2368734,"Neumont v. Monroe County, Florida",Neumont,"Monroe County, Florida",2003
5,2490,Paine,04-80543-CIV,2358425,Bazile v. Lucent Technologies,Bazile,Lucent Technologies,2005
6,2490,Paine,03-80590-CIV,2335767,Fick v. Metropolitan Life Insurance,Fick,Metropolitan Life Insurance,2004
7,2490,"Paine, Vitunac","99-10054-CIV-PAINE, 99-10054-CIV-VITUNAC",2312588,"Neumont v. Monroe County, Florida",Neumont,"Monroe County, Florida",2002
8,2490,"Paine, Lynch, Unite",0214160CIVPAINE Lynch,2297969,Wall v. Pennzoil-Quaker States Co.,Wall,Pennzoil-Quaker States Co.,2004


In [22]:
# Create a dataframe with just party names
columns_to_keep = ['party_1','party_2']
sample_dockets_reduced_v2 = sample_dockets_reduced_v1[columns_to_keep]

# Merge the two columns, drop duplicates and create a list
party_names = pd.concat([sample_dockets_reduced_v2['party_1'], sample_dockets_reduced_v2['party_2']])
party_names = party_names.drop_duplicates()
party_names = party_names.tolist()
print(f'The number of unique party names is {len(party_names)}')
print(party_names[:10])

The number of unique party names is 3002
['Neumont', 'Bazile', 'Fick', 'Wall', 'Sony Computer Entertainment Inc.', 'Information Resources, Inc.', 'Blanch', 'Dimich', 'Viacom International Inc.', 'Colgate-Palmolive Co.']


In [24]:
# Create a merged dataframe with judges and investments
# First, in the dockets file, have both party names in a single column for merging 
sample_dockets_reduced_v3 = sample_dockets_reduced_v1[['author_id','party_1','party_2','dateFiled_year']]

# Merge party names into a single column using melt
sample_dockets_reduced_v3 = sample_dockets_reduced_v1.melt(
    id_vars=['author_id','dateFiled_year'],
    value_vars=['party_1','party_2'],
    value_name='party_name')

# Reorder columns and drop duplicates
sample_dockets_reduced_v3 = sample_dockets_reduced_v3[['author_id','party_name','dateFiled_year']]
sample_dockets_reduced_v3 = sample_dockets_reduced_v3.drop_duplicates()
print(f'The shape of the melted dockets is {sample_dockets_reduced_v3.shape}')
sample_dockets_reduced_v3.head()

The shape of the melted dockets is (3706, 3)


,author_id,party_name,dateFiled_year
0,2490,Neumont,2003
1,2490,Bazile,2005
2,2490,Fick,2004
3,2490,Neumont,2002
4,2490,Wall,2004


In [9]:
# Now let's examine investments sample
sample_investments.head()

,person_id,disclosure_year,investment_id,inv_description
0,568,2015,1751139.0,Harris Bank IRA-Non Stock Account
1,568,2015,1751140.0,Harris Bank IRA-Non Stock Account
2,1816,2012,1565171.0,Bank of America
3,1816,2012,1565172.0,Bank of America
4,1816,2012,1565173.0,Bank of America


In [10]:
# Get a list of unique investment descriptions - these are strings we will try to match with
investment_descriptions = sample_investments['inv_description'].unique().tolist()
print(f'The number of unique investment descriptions is {len(investment_descriptions)}')
print(investment_descriptions)

The number of unique investment descriptions is 4102
['Harris Bank IRA-Non Stock Account', 'Bank of America', 'IRA #1 MORGAN STANLEY ("MS"): UBS FINANCIAL SERV., INC.', 'Ford Motor Credit Co. (Ser. Note)', 'General Motors ACB', 'DRU, INC', 'Caterpillar FNL SVC', 'Capital One Bank Glen Allen, VA CD', 'HSBC Fin. Corp.', 'BBTCop.T', 'GE Money Bank UT CD', 'M&T Capital Trust', 'PNC Capita) Trust E', 'Credit Suisse GU', 'J.P.Morgan Chase', 'American Express Credit', 'Capital Jumbo CD', 'Flagstone Bank CD', 'MORGAN STANLEY ACCOUNT: UBS FINANCIAL SERV. INC.', 'Duke Energy', 'Bristol Meyers Squibb', 'General Electric', 'Sunrise W & G', 'Hudson GO Purp B', 'Luzeme, COGOB', 'Texas Cap. Bk. CD', 'Kansas CY CLGS', 'DB Capiual T', 'Credit Suisse', 'Citigroup', 'Decutsche BK Cap Ill', 'UBS Bank USA', '1ll. Rural Bk Bd Rev Bond', 'Walgreen', 'Romeoville, IL GO', 'Wash. St. Higher Ed. Fac Bond', 'IM-IT Bond Fund 79 (Van Kampen)', 'TI. St. U. Rev. B', 'Rancho Mirage CAB', 'Harris Bank (checking)', 'Fed

In [25]:
# Keep relevant columns in investments data
sample_investments_reduced_v1 = sample_investments[['person_id','disclosure_year','inv_description']]
# Drop duplicates
sample_investments_reduced_v1 = sample_investments_reduced_v1.drop_duplicates()

# Rename and rearrange columns
sample_investments_reduced_v1 = sample_investments_reduced_v1.rename(columns={'person_id':'author_id','disclosure_year':'dateFiled_year'})
sample_investments_reduced_v1 = sample_investments_reduced_v1[['author_id','inv_description','dateFiled_year']]
print(f'The shape of the reduced investments is {sample_investments_reduced_v1.shape}')
sample_investments_reduced_v1.head()

The shape of the reduced investments is (8878, 3)


,author_id,inv_description,dateFiled_year
0,568,Harris Bank IRA-Non Stock Account,2015
2,1816,Bank of America,2012
6,1380,"IRA #1 MORGAN STANLEY (""MS""): UBS FINANCIAL SE...",2009
7,1380,Ford Motor Credit Co. (Ser. Note),2009
8,1380,General Motors ACB,2009


In [29]:
# Pipeline for string matching

stopwords = ['ltd','limited','company','inc','incorporated','corporation','corp','co','llc','plc','accounts','401K','bonds','']

def preprocess_names(names_list):
    """
    Preprocess a list of names by cleaning text (lowercase, strip spaces, remove special characters).
    
    Args:
        names_list (list): List of names to preprocess.
    
    Returns:
        list: Preprocessed list of names.
    """
    processed_list = []
    for name in names_list:
        # Convert to lowercase and remove unwanted characters
        if isinstance(name, str):  # Check if it's a string
            clean_name = name.lower().strip()
            # remove numerals but retain special characters
            clean_name = re.sub(r'\d+', '', clean_name)
            # remove stopwords
            clean_name = ' '.join([word for word in clean_name.split() if word not in stopwords])
            processed_list.append(clean_name)
        else:
            processed_list.append('')
    return processed_list

In [30]:
# Preprocess party names and investment descriptions
processed_party_names = preprocess_names(party_names)
processed_investment_descriptions = preprocess_names(investment_descriptions)

In [32]:
print(processed_party_names[:10])
print(processed_investment_descriptions[:10])

['neumont', 'bazile', 'fick', 'wall', 'sony computer entertainment inc.', 'information resources, inc.', 'blanch', 'dimich', 'viacom international inc.', 'colgate-palmolive co.']
['harris bank ira-non stock account', 'bank of america', 'ira # morgan stanley ("ms"): ubs financial serv., inc.', 'ford motor credit co. (ser. note)', 'general motors acb', 'dru,', 'caterpillar fnl svc', 'capital one bank glen allen, va cd', 'hsbc fin. corp.', 'bbtcop.t']


In [33]:
# Find the best match for each party name from a list of investment descriptions using cosine similarity

def find_best_match_cosine_all(party_names, investment_list):
    """
    Find the best match for each party name from a list of investment descriptions using cosine similarity.
    
    Args:
        party_names (list): List of party names to match.
        investment_list (list): List of investment descriptions.
    
    Returns:
        pd.DataFrame: DataFrame containing party name, best matching investment name, and best match score (cosine similarity).
    """
    # Start the timer
    start_time = time.time()
    print(f'Number of party names: {len(party_names)}')
    print(f'Number of investment descriptions: {len(investment_list)}')

    # Filter out party names with length < 5
    party_names = [name for name in party_names if len(name) >= 5]
    print(f'Number of party names after filtering: {len(party_names)}')

    # Combine all party names and investment descriptions for TF-IDF vectorization
    all_texts = party_names + investment_list
    print(f'Combined list of party names and investment descriptions has {len(all_texts)} items')
    
    # Convert texts to vectors using TF-IDF
    vectorizer = TfidfVectorizer().fit_transform(all_texts)
    vectors = vectorizer.toarray()
    print(f'Created TfidfVectors: the shape of the vectors is {vectors.shape}')

    # Split the vectorized results into two parts: party_names and investment_list vectors
    party_vectors = vectors[:len(party_names)]
    investment_vectors = vectors[len(party_names):]
    
    # Initialize list to store results
    results = []

    # Loop through each party name
    for i, party_name in enumerate(party_names):
        # Calculate cosine similarity between the current party_name and investment descriptions starting with the same letter
        pass # to be continued
    
        # Calculate cosine similarity between the current party_name vector and all investment vectors
        similarity_scores = cosine_similarity([party_vectors[i]], investment_vectors)[0]
        
        # Find the best match (highest score)
        best_match_idx = similarity_scores.argmax()
        best_match_score = similarity_scores[best_match_idx]
        best_match_name = investment_list[best_match_idx]
        # print(f'Best match for {party_name} is {best_match_name} with a score of {best_match_score}') - prints too many lines
        
        # Append results: party_name, best match, best match score
        results.append({
            "party_name": party_name,
            "best_match": best_match_name,
            "best_match_score": best_match_score
        })
    
    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)

    # End the timer
    end_time = time.time()
    print(f'Finished in {end_time - start_time:.2f} seconds')
    return results_df

In [18]:
# Find best match for each party name using cosine similarity
similarity_matching_cosine = find_best_match_cosine_all(processed_party_names, processed_investment_descriptions)

Combined list of party names and investment descriptions has 7104 items
Created TfidfVectors: the shape of the vectors is (7104, 6568)
Finished in 376.52 seconds


In [19]:
# Display the results and save to a CSV file for manual review
print(similarity_matching_cosine)

                           party_name                        best_match  \
0                             neumont  harris bank iranon stock account   
1                              bazile  harris bank iranon stock account   
2                                fick  harris bank iranon stock account   
3                                wall  harris bank iranon stock account   
4     sony computer entertainment inc             computer services inc   
...                               ...                               ...   
2997                    aegon usa inc                      ubs bank usa   
2998                         fiorella  harris bank iranon stock account   
2999                             mais  harris bank iranon stock account   
3000                           postel  harris bank iranon stock account   
3001     the mcgrawhill companies inc                     mcgrawhill co   

      best_match_score  
0             0.000000  
1             0.000000  
2             0.000000  

In [20]:
# Keep the matches with a best_match_score >= 0.5 
similarity_matching_cosine_filtered = similarity_matching_cosine[similarity_matching_cosine['best_match_score'] >= 0.5]
print(similarity_matching_cosine_filtered.head())

                        party_name              best_match  best_match_score
8         viacom international inc              viacom inc          0.857042
15  credit suisse first boston llc           credit suisse          0.686442
37                   united states  united states cellular          0.672540
60              electric insurance        general electric          0.561760
61                           allen                allen go          0.741620


In [33]:
# Save the filtered results to a CSV file for manual review
similarity_matching_cosine_filtered.to_csv('similarity_matching_cosine_similarity_filtered.csv', index=False)

In [34]:
# Merge investments and reduced dockets data on author_id and dateFiled_year
merged_dock_inv_str_matching = pd.merge(sample_dockets_reduced_v3, sample_investments_reduced_v1, on=['author_id','dateFiled_year'], how='inner')
print(f'The shape of the merged data is {merged_dock_inv_str_matching.shape}')
merged_dock_inv_str_matching.head()

The shape of the merged data is (89197, 4)


,author_id,party_name,dateFiled_year,inv_description
0,2490,Neumont,2003,BANK ACCOUNTS
1,2490,Neumont,2003,Wachovia Bank
2,2490,Neumont,2003,"West Palm Beach, Florida"
3,2490,Neumont,2003,Fidelity Fed. Bk & Tr
4,2490,Neumont,2003,COMMON STOCK


In [35]:
# Preprocess party_name and inv_description in the merged dataframe
merged_dock_inv_str_matching['processed_party_name'] = preprocess_names(merged_dock_inv_str_matching['party_name'].tolist())
merged_dock_inv_str_matching['processed_inv_description'] = preprocess_names(merged_dock_inv_str_matching['inv_description'].tolist())

In [36]:
# Examine merged_dock_inv_str_matching
merged_dock_inv_str_matching.head()

,author_id,party_name,dateFiled_year,inv_description,processed_party_name,processed_inv_description
0,2490,Neumont,2003,BANK ACCOUNTS,neumont,bank accounts
1,2490,Neumont,2003,Wachovia Bank,neumont,wachovia bank
2,2490,Neumont,2003,"West Palm Beach, Florida",neumont,"west palm beach, florida"
3,2490,Neumont,2003,Fidelity Fed. Bk & Tr,neumont,fidelity fed. bk & tr
4,2490,Neumont,2003,COMMON STOCK,neumont,common stock


In [50]:
# Examine the excel file with matched results
similarity_matching_cosine_manual = pd.read_excel('similarity_matching_cosine_similarity_filtered.xlsx')

# Filter results where 'keep' column equals 1
similarity_matching_cosine_manual_filtered = similarity_matching_cosine_manual[similarity_matching_cosine_manual['keep'] == 1]

# Drop the 'keep' column
similarity_matching_cosine_manual_filtered = similarity_matching_cosine_manual_filtered.drop(columns='keep')
print(f'The shape of the filtered manual results is {similarity_matching_cosine_manual_filtered.shape}')
similarity_matching_cosine_manual_filtered.head()

The shape of the filtered manual results is (141, 3)


,party_name,best_match,best_match_score
1,caterpillar inc,caterpillar inc,1.0
2,microsoft corp,microsoft corp,1.0
3,ford,ford f,1.0
4,autozone inc,autozone inc,1.0
5,citigroup inc,citigroup inc,1.0


In [51]:
# Merge dataframes based on matching party names and best matches, and also keep = 1
df_result = pd.merge(merged_dock_inv_str_matching, similarity_matching_cosine_manual_filtered,
                     left_on=['processed_party_name', 'processed_inv_description'],
                     right_on=['party_name', 'best_match'],
                     how='left')

# Fill NaN values in the 'keep' column with 0 for non-matches
df_result.head()

,author_id,party_name_x,dateFiled_year,inv_description,processed_party_name,processed_inv_description,party_name_y,best_match,best_match_score
0,2490,Neumont,2003,BANK ACCOUNTS,neumont,bank accounts,NaN,NaN,NaN
1,2490,Neumont,2003,Wachovia Bank,neumont,wachovia bank,NaN,NaN,NaN
2,2490,Neumont,2003,"West Palm Beach, Florida",neumont,west palm beach florida,NaN,NaN,NaN
3,2490,Neumont,2003,Fidelity Fed. Bk & Tr,neumont,fidelity fed bk tr,NaN,NaN,NaN
4,2490,Neumont,2003,COMMON STOCK,neumont,common stock,NaN,NaN,NaN
